In [1]:
# --- choose one installation method if required ---
# Standard pip (Jupyter desktop / server)
# !pip install folium pandas

# For JupyterLite / browser environments using piplite, uncomment:
# import piplite
# await piplite.install(['folium','pandas'])

# --- imports ---
import folium
import pandas as pd
import requests
import io
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

In [2]:
# Download the CSV (lab-provided URL)
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
r = requests.get(URL, timeout=20)
r.raise_for_status()
spacex_df = pd.read_csv(io.BytesIO(r.content))

# Keep the columns the lab uses (rename if your file uses different names)
print("Columns in CSV:", spacex_df.columns.tolist())

# Keep relevant subset (adjust names if they differ)
spacex_df = spacex_df.rename(columns=lambda c: c.strip())
spacex_df = spacex_df[['Launch Site','Lat','Long','class','Flight Number','Date']] \
    .rename(columns={'class':'Class','Lat':'Lat','Long':'Long','Flight Number':'FlightNumber'})

spacex_df.head()

Columns in CSV: ['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site', 'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome', 'class', 'Lat', 'Long']


,Launch Site,Lat,Long,Class,FlightNumber,Date
0,CCAFS LC-40,28.562302,-80.577356,0,1,2010-06-04
1,CCAFS LC-40,28.562302,-80.577356,0,2,2010-12-08
2,CCAFS LC-40,28.562302,-80.577356,0,3,2012-05-22
3,CCAFS LC-40,28.562302,-80.577356,0,4,2012-10-08
4,CCAFS LC-40,28.562302,-80.577356,0,5,2013-03-01


In [3]:
launch_sites_df = spacex_df.groupby('Launch Site', as_index=False).first()[['Launch Site','Lat','Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
# Start map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a marker + circle for each launch site
for idx, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    coord = [row['Lat'], row['Long']]
    # circle
    folium.Circle(
        location=coord,
        radius=50000,            # radius in meters (tune as desired)
        color='#3186cc',
        fill=True,
        fill_opacity=0.2,
        popup=folium.Popup(site_name, parse_html=True)
    ).add_to(site_map)
    # label marker (small text label)
    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size:12px; color:#3186cc;"><b>{site_name}</b></div>',
        )
    ).add_to(site_map)

# display map
site_map

In [6]:
# Start map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a marker + circle for each launch site
for idx, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    coord = [row['Lat'], row['Long']]
    # circle
    folium.Circle(
        location=coord,
        radius=50000,            # radius in meters (tune as desired)
        color='#3186cc',
        fill=True,
        fill_opacity=0.2,
        popup=folium.Popup(site_name, parse_html=True)
    ).add_to(site_map)
    # label marker (small text label)
    folium.map.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size:12px; color:#3186cc;"><b>{site_name}</b></div>',
        )
    ).add_to(site_map)

# display map
site_map

In [7]:
# Add mouse position display to the map (shows Lat, Long when you hover)
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map

In [8]:
# Haversine-based distance (km)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in km
    lat1_r, lon1_r, lat2_r, lon2_r = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2_r - lon1_r
    dlat = lat2_r - lat1_r
    a = sin(dlat / 2)**2 + cos(lat1_r) * cos(lat2_r) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Example: compute distance from a launch site to a coastline point you chose with MousePosition.
# Replace these example coordinates with the coastline coordinate you picked.

# Example: Kennedy Space Center approximate coastline point (you should pick via MousePosition)
# Example launch site: 'CCAFS SLC-40' (you can change to one from launch_sites_df)
example_site = 'CCAFS SLC-40'
site_row = launch_sites_df[launch_sites_df['Launch Site'] == example_site].iloc[0]
launch_lat, launch_lon = site_row['Lat'], site_row['Long']

# Example coastline coordinate (replace with the coordinates you get from MousePosition)
coast_lat, coast_lon = 28.56367, -80.57163  # example: near Cape Canaveral coastline

distance_km = calculate_distance(launch_lat, launch_lon, coast_lat, coast_lon)
print(f"Distance from {example_site} to coastline point ({coast_lat}, {coast_lon}): {distance_km:.2f} km")

# Add a marker for the coastline point and a polyline from launch site -> coastline
coast_marker = folium.map.Marker(
    [coast_lat, coast_lon],
    icon=DivIcon(icon_size=(150,36), icon_anchor=(0,0),
                 html=f'<div style="font-size:12px;color:#d35400;"><b>{distance_km:.2f} km</b></div>')
)
site_map.add_child(coast_marker)

# PolyLine between launch site and coastline
line = folium.PolyLine(locations=[[launch_lat, launch_lon], [coast_lat, coast_lon]], weight=3, color='blue')
site_map.add_child(line)

# Add a small pop-up marker for the launch site (if not already labeled)
folium.Marker([launch_lat, launch_lon], popup=f"{example_site} (launch site)").add_to(site_map)

site_map

Distance from CCAFS SLC-40 to coastline point (28.56367, -80.57163): 0.51 km


In [9]:
# Example list of candidate coords (replace with points you recorded)
candidate_points = [
    (28.570, -80.583),  # candidate 1
    (28.56367, -80.57163),  # candidate 2
    (28.600, -80.640)   # candidate 3
]

def find_closest(launch_lat, launch_lon, candidates):
    best = None
    best_dist = float('inf')
    for lat, lon in candidates:
        d = calculate_distance(launch_lat, launch_lon, lat, lon)
        if d < best_dist:
            best_dist = d
            best = (lat, lon, d)
    return best

closest = find_closest(launch_lat, launch_lon, candidate_points)
print("Closest candidate to", example_site, "->", closest)
# Draw it on the map (if desired)
if closest:
    latc, lonc, dk = closest
    folium.Marker([latc, lonc], popup=f"Closest candidate: {dk:.2f} km").add_to(site_map)
    folium.PolyLine(locations=[[launch_lat, launch_lon], [latc, lonc]], color='red', weight=2, dash_array='5').add_to(site_map)
    site_map

Closest candidate to CCAFS SLC-40 -> (28.56367, -80.57163, 0.5095839932574941)


In [10]:
site_map.save('spacex_launch_sites_map.html')
print("Saved map to spacex_launch_sites_map.html")

Saved map to spacex_launch_sites_map.html
